# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [90]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [91]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [92]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [93]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [94]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [95]:
# Create a Keyspace 
create_keyspace = """CREATE KEYSPACE IF NOT EXISTS sparkifydatabase
                with REPLICATION = { 
                    'class' : 'SimpleStrategy', 'replication_factor' : 1 
                    }
"""
try:
    session.execute(create_keyspace)
except Exception as e:
    print(e)    

#### Set Keyspace

In [96]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("sparkifydatabase")
except Excepion as e:
    print(e)

### Query #1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Create a table to execute above query# 1.

In [97]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query_to_create_table1 = """CREATE TABLE IF NOT EXISTS session_items (
                                artist TEXT, song TEXT, length FLOAT, sessionId INT, itemInSession INT, 
                                PRIMARY KEY (sessionId, itemInSession))
                        """
try: 
    session.execute(query_to_create_table1)
except Exception as e:
    print(e)                    

#### Insert the data into the table

In [98]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_items (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[10], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [99]:
## Add in the SELECT statement to verify the data was entered into the table
query_to_select_table1 = "SELECT artist, song, length FROM  session_items "
query_to_select_table1 = query_to_select_table1 + "WHERE sessionId = %s AND itemInSession = %s"
try:
    rows = session.execute(query_to_select_table1, (338, 4))
except Exception as e:
    print(e)

# Use the pretty table and assign headings in that table
results = PrettyTable(['Artist', 'Song', 'Length'])

for row in rows:
    results.add_row([row.artist, row.song, row.length])
print(results)

+-----------+------+--------------------+
|   Artist  | Song |       Length       |
+-----------+------+--------------------+
| Faithless |  50  | 495.30731201171875 |
+-----------+------+--------------------+


### Query #2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create the table to execute query #2. Here, we used userId, sessionId and itemInSession as primary key where (userId and sessionId) are partition key and itemInSession is clustering column. Also we used the clause "WITH CLUSTERING ORDER BY" in order to sort the data in the column.

In [100]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query_to_create_table2 = """CREATE TABLE IF NOT EXISTS user_details (
                                userId INT, sessionId INT, artist TEXT, song TEXT, firstName TEXT, lastName TEXT, itemInSession INT, 
                                PRIMARY KEY ((userId, sessionId), itemInSession))
                                WITH CLUSTERING ORDER BY (itemInSession ASC);
                        """

try: 
    session.execute(query_to_create_table2)
except Exception as e:
    print(e)                  

#### Insert the data into the table

In [101]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_details (userId, sessionId, artist, song, firstName, lastName, itemInSession) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [102]:
# Add in the SELECT statement to verify the data was entered into the table
query_to_select_table2 = "SELECT artist, song, firstName, lastName FROM user_details "
query_to_select_table2 = query_to_select_table2 + "WHERE userId = %s AND sessionId = %s;"
try:
    rows = session.execute(query_to_select_table2, (10, 182))
except Exception as e:
    print(e)

# Use the pretty table and assign headings in that table
results = PrettyTable(['Artist', 'Song', 'First Name', 'Last Name'])

for row in rows:
    results.add_row([row.artist, row.song, row.firstname, row.lastname])

print(results)

+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


### Query #3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create a table to execute query #3. Columns song and userId are used as Primary Key where song is the partition key and userId is the clustering column.

In [103]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query_to_create_table3 = """CREATE TABLE IF NOT EXISTS user_song (
                                song TEXT, userId INT, firstName TEXT, lastName TEXT,
                                PRIMARY KEY (song, userId)
                        )"""
try: 
    session.execute(query_to_create_table3)
except Exception as e:
    print(e)                    

#### Insert the data into the table

In [104]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song (song, userId, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s) "
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [105]:
# SELECT statement to verify the data was entered into the table
query_to_select_table3 = "SELECT song, firstName, lastName FROM user_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query_to_select_table3)
except Exception as e:
    print(e)

# Use the pretty table and assign headings in that table
results = PrettyTable(['Song', 'First Name', 'Last Name'])
    
for row in rows:
    results.add_row([row.song, row.firstname, row.lastname])
    
print(results)

+---------------------------+------------+-----------+
|            Song           | First Name | Last Name |
+---------------------------+------------+-----------+
| All Hands Against His Own | Jacqueline |   Lynch   |
| All Hands Against His Own |   Tegan    |   Levine  |
| All Hands Against His Own |    Sara    |  Johnson  |
+---------------------------+------------+-----------+


### Drop the tables before closing out the sessions

In [106]:
## Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS sparkifydatabase.session_items")
session.execute("DROP TABLE IF EXISTS sparkifydatabase.user_details")
session.execute("DROP TABLE IF EXISTS sparkifydatabase.user_song")

### Close the session and cluster connection¶

In [107]:
session.shutdown()
cluster.shutdown()